In [1]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.warn("deprecated", DeprecationWarning)
from scipy.spatial import distance as dist
from collections import OrderedDict
from OurClasses import *
from JServer import *
import math

/var/folders/yw/ky3wyxtn5wgbtrvvv7s7trgc0000gn/T/ipykernel_87729/2667076721.py:6: DeprecationWarning: deprecated
  warnings.warn("deprecated", DeprecationWarning)


In [2]:
def video_to_array(path):
    cap = cv.VideoCapture(path)
    frameCount = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    buf = np.empty((frameCount, 400, 640, 3), np.dtype('uint8'))

    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        buf[fc] = cv.resize(frame, (640, 400))
        fc += 1

    cap.release()
    return buf


        
    

In [3]:
def filter_with_stats(img,area_th=300):
    output = cv.connectedComponentsWithStats(img, 8, cv.CV_32S)
    (numLabels, labels, stats, centroids) = output
    for i in range(0, numLabels):
        x = stats[i, cv.CC_STAT_LEFT]
        y = stats[i, cv.CC_STAT_TOP]
        w = stats[i, cv.CC_STAT_WIDTH]
        h = stats[i, cv.CC_STAT_HEIGHT]
        area= stats[i, cv.CC_STAT_AREA]
        if area < area_th:
            labels[labels == i] = 0
    labels[labels != 0] = 255
    return labels.astype(np.uint8) 

def ball_filter(img,back_object):
    lab = cv.cvtColor(img,cv.COLOR_RGB2LAB)
    fgMask = back_object.apply(lab[:,:,1])
    th, b_th = cv.threshold(lab[:,:,2], 100, 255, cv.THRESH_BINARY_INV)
    th,a_th = cv.threshold(fgMask, 180, 255, cv.THRESH_BINARY)
    a_filter = filter_with_stats(a_th,200)
    b_filter = filter_with_stats(b_th,200)
    th_img = cv.bitwise_and(a_filter,b_filter)
    return th_img

def ball_detection(img,back_object):
    balls_lst = []
    th_img = ball_filter(img,back_object)
    contours, _ = cv.findContours(th_img.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # list for storing names of shapes
    for contour in contours:
        # approx = cv.approxPolyDP(contour, 0.01 * cv.arcLength(contour, True), True)
        #     # finding center point of shapes
        M = cv.moments(contour)
        if M['m00'] != 0.0:
            x = int(M['m10']/M['m00'])
            y = int(M['m01']/M['m00'])
    
        #     # putting shape name at center of each shape
            # if len(approx)>12:
            #     cv.putText(img, 'circle', (x, y),cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            ball = Ball(x,y,cv.arcLength(contour,True)) 
            balls_lst.append(ball)
    return th_img,balls_lst,contours

In [4]:
# def get_closest_point(ball_old,ball_new):
#     old = np.array([(i.x,i.y) for i in ball_old])
#     new = np.array([(j.x,j.y) for j in ball_new])
#     D = dist.cdist(old,new)
#     if len(old) > len(new):
#         D=D.T
#     return[(np.min(D[row]),np.argmin(D[row])) for row in range(D.shape[0])]
def get_closest_point(ball_old,ball_new):
    old = np.array([(i.x,i.y) for i in ball_old])
    new = np.array([(j.x,j.y) for j in ball_new])
    D = dist.cdist(old,new)
    arr = np.column_stack((np.arange(len(old)),np.zeros((len(old),2))))
    if D.shape[0]>D.shape[1]:
        for dis in range(D.shape[1]):
            d = np.unravel_index(D.argmin(), D.shape)
            arr[d[0],1] = d[1]
            arr[d[0],2] = D[d]
            D[d[0]] = 999
            D[:,d[1]] = 999
    else:
        for dis in range(D.shape[0]):
            d = np.unravel_index(D.argmin(), D.shape)
            arr[d[0],1] = d[1]
            arr[d[0],2] = D[d]
            D[d[0]] = 999
            D[:,d[1]] = 999
    arr[(arr[:,2]==0),2]=-1
    return arr

In [5]:
def draw_label(img, text, pos, bg_color):
   font_face = cv.FONT_HERSHEY_SIMPLEX
   scale = 2
   color = (0, 0, 0)
   thickness = cv.FILLED
   margin = 2
   txt_size = cv.getTextSize(text, font_face, scale, thickness)

   end_x = pos[0] + txt_size[0][0] + margin
   end_y = pos[1] - txt_size[0][1] - margin

   cv.rectangle(img, pos, (end_x, end_y), bg_color, thickness)
   cv.putText(img, text, pos, font_face, scale, color, 1, cv.LINE_AA)

In [6]:
# images = video_to_array('regular2.mp4')
images = video_to_array('library1.mp4')


In [18]:

colors = [(255,0,0),[0,255,0],(0,0,255),[255,255,255]]
backSub = cv.createBackgroundSubtractorMOG2()
backSub2 = cv.createBackgroundSubtractorMOG2()#y
historyReal=History(999,4)

ballsCounter=0
maxDistanceToMatch = 180
videoTrail = cv.VideoCapture("fireball.gif")
frameT_width = int(videoTrail.get(3))
frameT_height = int(videoTrail.get(4))

myServer = JServer()
for i in range(20):
        historyReal.insertAllBalls([Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL)],Btype.REAL)

for count,img in enumerate(images):
    copy_img = img.copy()
    th_img, balls_list  , contours = ball_detection(img,backSub)
    hands_mask  = hands_detection(img, backSub2, detect = False, ball = None) #y

    balls_list = [ball for ball in balls_list if ball.area>50]
    # print("len " + str(len(balls_list)),end='')
    # for b in balls_list:
    #     print("x " + str(b.x),end='')
    #     print("y " + str(b.y),end='')
    #     print("area " + str(b.area))


    ballsAlive , aliveIDs,ballsDead,deadIDs, numOfBalls = historyReal.predictNumOfBalls(historySizeNotReal=20,historySizeSpeedKill=20,historySizeSpeedRevive=10,historySizeSizeRevive=10,THavgNotRealKill=0.85,THavgSpeedToKill=10,THavgSpeedToRevive=20,THavgSizeToRevive=70)
    #kalman~~~~~~~~~~
    for idx,id in enumerate(aliveIDs):
        if(ballsAlive[idx].btype==Btype.REAL):
            ball = historyReal.get(id).kalmanPredict()
            ballsAlive[idx].x = ball.x
            ballsAlive[idx].y = ball.y
            ballsAlive[idx].btype = Btype.KALMAN_PREDICTION
    #kalman end~~~~~~
    lst=[]
    #~~~~alive balls~~~~~
    if len(ballsAlive)>0:
        lst = get_closest_point(ballsAlive,balls_list) #lst[i][0]=old balls index,lst[i][1]=new balls index,lst[i][2]=distance
        historyReal.insertByLst(lst,aliveIDs,balls_list,img,backSub2,useHand = True,maxDistanceToMatch=maxDistanceToMatch)
    #~~~~dead balls~~~~~
    if len(balls_list)>len(ballsAlive):
        indices = [int(lst[i][1]) for i in range(len(lst))]
        balls_list = np.delete(balls_list, indices)
        lst_dead = get_closest_point(ballsDead,balls_list)
        historyReal.insertByLst(lst_dead,deadIDs,balls_list,img,backSub2,useHand = False)




    # print(numOfBalls)
    if historyReal.isNewBallAtMax():
        ballsCounter+=1


    historyReal.drawLastBalls(copy_img)
    #trail part
    retT, frameT = videoTrail.read()
    if not retT:
        # print('no video')
        videoTrail.set(cv.CAP_PROP_POS_FRAMES, 0)
        retT, frameT = videoTrail.read()
        # continue


    historyReal.drawTrailsLastBalls(copy_img,640,400,frameT,frameT_width,frameT_height)

    draw_label(copy_img, str(ballsCounter), (20,60), (255, 0, 0))
    # print(numOfBalls)
    myServer.sendData(copy_img)
    # cv.imshow('first_mask',th_img)
    # cv.imshow('hands_mask',hands_mask)
    # cv.imshow('real_time',copy_img)
    # cv.waitKey(0)
cv.destroyAllWindows()


Socket successfully created


OSError: [Errno 48] Address already in use

In [14]:
myServer.closeConnection()

In [12]:
    # for contour in contours:
    #     cv.drawContours(image=copy_img, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.16.103.230:8080
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/bar/Desktop/studies/semester7/imageProcessing/finalProject/InteractiveJuggling/venv2/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/bar/Desktop/studies/semester7/imageProcessing/finalProject/InteractiveJuggling/venv2/lib/python3.9/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Users/bar/Desktop/studies/semester7/imageProcessing/finalProject/InteractiveJuggling/venv2/lib/python3.9/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/bar/Desktop/studies/semester7/imageProcessing/finalProject/InteractiveJuggling/venv2/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 678, i

SystemExit: 1

In [11]:
# define a video capture object
colors = [(255,0,0),[0,255,0],(0,0,255),[255,255,255]]
backSub = cv.createBackgroundSubtractorMOG2()
backSub2 = cv.createBackgroundSubtractorMOG2()#y
historyReal=History(999,4)

ballsCounter=0
maxDistanceToMatch = 180

vid = cv.VideoCapture(0)

for i in range(20):
        historyReal.insertAllBalls([Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL),Ball(0,0,0,Btype.REAL)],Btype.REAL)

while(True):

    # Capture the video frame
    # by frame
    ret, img = vid.read()
    copy_img = img.copy()
    th_img, balls_list  , contours = ball_detection(img,backSub)
    hands_mask  = hands_detection(img, backSub2, detect = False, ball = None) #y

    balls_list = [ball for ball in balls_list if ball.area>50]
    # print("len " + str(len(balls_list)),end='')
    # for b in balls_list:
    #     print("x " + str(b.x),end='')
    #     print("y " + str(b.y),end='')
    #     print("area " + str(b.area))


    ballsAlive , aliveIDs,ballsDead,deadIDs, numOfBalls = historyReal.predictNumOfBalls(historySizeNotReal=20,historySizeSpeedKill=20,historySizeSpeedRevive=10,historySizeSizeRevive=10,THavgNotRealKill=0.85,THavgSpeedToKill=10,THavgSpeedToRevive=20,THavgSizeToRevive=70)
    #kalman~~~~
    for idx,id in enumerate(aliveIDs):
        if(ballsAlive[idx].btype==Btype.REAL):
            ball = historyReal.get(id).kalmanPredict()
            ballsAlive[idx].x = ball.x
            ballsAlive[idx].y = ball.y
            ballsAlive[idx].btype = Btype.KALMAN_PREDICTION
    #kalman end~~~~
    lst=[]
    #~alive balls~~


    if len(ballsAlive)>0 and len(balls_list)>0:
        print(f'cond1 : alive -{len(ballsAlive)} , detections -{len(balls_list)} ')
        lst = get_closest_point(ballsAlive,balls_list) #lst[i][0]=old balls index,lst[i][1]=new balls index,lst[i][2]=distance
        historyReal.insertByLst(lst,aliveIDs,balls_list,img,backSub2,useHand = True,maxDistanceToMatch=maxDistanceToMatch)
        indices = [int(lst[i][1]) for i in range(len(lst))]
        balls_list = np.delete(balls_list, indices)
    #~dead balls~~
    if len(balls_list)> 0 and len(ballsDead)>0:
        print(f'cond1 : deat -{len(ballsDead)} , detections -{len(balls_list)} ')
        lst_dead = get_closest_point(ballsDead,balls_list)
        historyReal.insertByLst(lst_dead,deadIDs,balls_list,img,backSub2,useHand = False)




    # print(numOfBalls)
    if historyReal.isNewBallAtMax():
        ballsCounter+=1


    historyReal.drawLastBalls(copy_img)
    #trail part
    # retT, frameT = videoTrail.read()
    # if not retT:
    #     # print('no video')
    #     videoTrail.set(cv.CAP_PROP_POS_FRAMES, 0)
    #     retT, frameT = videoTrail.read()
    #     # continue


    # historyReal.drawTrailsLastBalls(copy_img,640,400,frameT,frameT_width,frameT_height)

    draw_label(copy_img, str(ballsCounter), (20,60), (255, 0, 0))
    # print(numOfBalls)
    # cv.imshow('first_mask',th_img)
    # cv.imshow('hands_mask',hands_mask)
    # cv.imshow('real_time',copy_img)
    # cv.waitKey(0)

    # Display the resulting frame
    cv.imshow('frame', copy_img)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv.destroyAllWindows()

cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : deat -4 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -3 
cond1 : deat -3 , detections -2 
cond1 : alive -1 , detections -4 
cond1 : deat -3 , detections -3 
cond1 : alive -1 , detections -2 
cond1 : deat -3 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -2 
cond1 : deat -3 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detections -1 
cond1 : alive -1 , detect

KeyboardInterrupt: 